In [4]:
import cvxpy as cp
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
## PROBLEM 3
# Create data
m = 30
n = 10
np.random.seed(2021)
A = np.random.randn(m, n)
y = np.random.randn(m)

# Set up CVX problem
x = cp.Variable(n)
objective = cp.Minimize(0.5*cp.norm(A@x - y,2)**2)
constraints = [0 <= x]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`
result = prob.solve()
# The optimal value for x is stored in `x.value`
xstar = x.value
# The optimal Lagrange multiplier for a constraint is stored in
# `constraint.dual_value`
lambdastar = constraints[0].dual_value
    

Q = A.T @ A
b = A.T @ y
Q_inv = np.linalg.inv(Q)

# check for optimality
print(np.linalg.norm((Q @ xstar - b) - lambdastar))


tol = 1e-6
maxiter = 10000

# Primal-Dual method
lambdak = np.zeros(n)
xk = np.zeros(n)
k = 0
while (np.linalg.norm(Q @ xk - b - lambdak) > tol ) and (k < maxiter):
    T = (lambdak == 0)
    Asub = A[:,T]
    Qsub = Asub.T @ Asub
    bsub = Asub.T @ y
    xk_sub = np.linalg.inv(Qsub) @ bsub
    xk = np.zeros(n)
    xk[T] = xk_sub
    xk[xk<0] = 0
    
    lambdak = Q @ xk - b
    lambdak[lambdak<tol] = 0
    k = k+1

print(np.linalg.norm((Q @ xk - b) - lambdak))
print(k)


# Version 2 : projected gradient descent
xk = np.zeros(n)
lambdak = np.zeros(n)
k = 0
alpha = 0.009
while (np.linalg.norm(Q @ xk - b - lambdak) > tol ) and (k < maxiter):
    r = b - Q @ xk
    xk = xk + alpha*r
    xk[xk<0] = 0
    lambdak = Q @ xk - b
    lambdak[lambdak<0] = 0
    k = k+1
    
print(np.linalg.norm((Q @ xk - b) - lambdak))
print(k)